In [16]:
import ee
ee.Authenticate()
ee.Initialize()
#print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())
import geemap
import folium


Successfully saved authorization token.


### Long Beach Ship Detection

In [7]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = ee.Image(ee.ImageCollection('LANDSAT/LC8_L1T_8DAY_TOA').first())

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, zoom=10)  # LA
Map.addLayer(ndwi, ndwiViz, 'NDWI', False)

# Display the map
Map


Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

### Saving Image

In [34]:
ee.Image("LANDSAT/LC08/C01/T1/LC08_044034_20140318").select('B2').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32610',
 'transform': [30, 0, 460785, 0, -30, 4264215]}

In [35]:
raw = ee.Image("LANDSAT/LC08/C01/T1/LC08_044034_20140318").select("B4","B3","B2")
coord = ee.Geometry.Rectangle(33.75599777953796, -118.28713097440811,33.69432004321109, -118.16404976872677)
projection = raw.select('B2').projection().getInfo()

In [43]:
task = ee.batch.Export.image.toDrive(**{
    "image":raw,
    "description": "test",
    "folder" :"GEE_Images",
    "scale" : 30,
    "region" : coord.getInfo()["coordinates"],
    "maxPixels": 1e12
})

In [44]:
task.start()

In [45]:
coordUCSD = ee.Geometry.Rectangle(32.88184783597174, -117.23838846877605,32.880478306981615, -117.23652165132586)

In [50]:
task2 = ee.batch.Export.image.toDrive(**{
    "image":raw,
    "description": "testUCSD",
    "folder" :"GEE_Images",
    "scale" : 30,
    "region" : coordUCSD.getInfo()["coordinates"],
    "maxPixels": 1e12
})

In [51]:
task2.start()

In [59]:
taskAsset = ee.batch.Export.image.toAsset(**{
    "image":raw,
    "description": "testUCSDAsset",
    "scale" : 30,
    "region" : coordUCSD.getInfo()["coordinates"],
    "maxPixels": 1e12,
    "assetId": 'projects/alexmak/assets/testUCSD'
})

In [60]:
taskAsset.start()